In [4]:
def gen_rnn_data(n):
    all_text = ""
    with open('../data/shakespeare.txt') as inp:
        for line in inp:
            if len(line.split()) < 3:
                continue
            all_text += line[:-1]
        seqs = []
        for i in range(40, len(all_text), n):
            seqs.append(all_text[i-40:i+1])
        return seqs


In [5]:
print(gen_rnn_data(50))

['From fairest creatures we desire increase', "reby beauty's rose might never die,But as", 'r should by time decease,His tender heir ', 'r his memory:But thou contracted to thine', "ht eyes,Feed'st thy light's flame with se", 'ntial fuel,Making a famine where abundanc', 'y self thy foe, to thy sweet self too cru', "hat art now the world's fresh ornament,An", 'rald to the gaudy spring,Within thine own', "est thy content,And tender churl mak'st w", 'iggarding:  Pity the world, or else this ', "e,  To eat the world's due, by the grave ", 'When forty winters shall besiege thy brow', "deep trenches in thy beauty's field,Thy y", 'oud livery so gazed on now,Will be a tatt', ' of small worth held:Then being asked, wh', 'hy beauty lies,Where all the treasure of ', ' days;To say within thine own deep sunken', 'e an all-eating shame, and thriftless pra', "uch more praise deserved thy beauty's use", "couldst answer 'This fair child of mineSh", "y count, and make my old excuse'Proving h", ' by succ